In [5]:
# importing the libraries
import re
import nltk
import string
import wikipedia
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import aiml

In [6]:
def details(topic):
    page_object = wikipedia.page(topic)
    my_article = page_object.content

    corpus = '' # one big string !! 
    
    # This for loop loops for every word present in the corpus
    for words in my_article:
        corpus += words
    
    # we lower case it here (we could do it upper case too)
    corpus = corpus.lower()

    corpus = re.sub(r'\[[0-9]*\]', ' ', corpus) 
    corpus = re.sub(r'\s+', ' ', corpus)
    corpus = re.sub(r'\=+', '', corpus)
    
    
    sent_tokens = nltk.sent_tokenize(corpus)
    word_tokens = nltk.word_tokenize(corpus)

    lemma = nltk.stem.WordNetLemmatizer()

    def lemtokens(tokens):
        return [lemma.lemmatize(token) for token in tokens]
    dict_remove_punctuations = dict((ord(punct), None) for punct in string.punctuation)

    def lemnormalize(text):
        return lemtokens(nltk.word_tokenize(text.lower().translate(dict_remove_punctuations)))


    # ## Response Generation 


    def response(user_response):
        robot_response = ''
        TfidfVec = TfidfVectorizer(tokenizer=lemnormalize, stop_words='english')
        tfidf = TfidfVec.fit_transform(sent_tokens)
        vals = cosine_similarity(tfidf[-1],tfidf)
        idx = vals.argsort()[0][-2]
        flat = vals.flatten()
        flat.sort()
        req_tfidf = flat[-2]
        if(req_tfidf == 0):
            robot_response = robot_response + 'I am Sorry! I couldn\'t get it'
            return robot_response
        else:
            robot_response = robot_response + sent_tokens[idx]
            return robot_response 


    # ## Defining a convo

    flag = True
    print('Bot: My name is Bot. Lets try to get to know about {} in detail, If you want to exit anytime, just type \'bye\''.format(topic))
    while(flag == True):
        user_response = input()
        user_response = user_response.lower()
        if(user_response != 'bye'):
            if(user_response == 'thanks' or user_response == 'thank you'):
                flag = False
                print('Bot: You are welcome')
            else:

                sent_tokens.append(user_response)
                word_tokens = word_tokens + nltk.word_tokenize(user_response)
                final_words = list(set(word_tokens))
                print('Bot: ', end='')
                print(response(user_response))
                sent_tokens.remove(user_response)
        else:
            flag = False
            print('Bot: Goodbye! Take care :))')

In [8]:
details('Android Operating System')

Bot: My name is Bot. Lets try to get to know about Android Operating System in detail, If you want to exit anytime, just type 'bye'
first android 
Bot: 

G:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


the security issue is only with android 8 (android oreo) and android 9 (android pie).
phones android
Bot: for example, as android was developed initially as a phone os, hardware such as microphones were required, while over time the phone function became optional.
exit
Bot: I am Sorry! I couldn't get it
bye
Bot: Goodbye! Take care :))


In [9]:

"""
Basic chatbot design --- for your own modifications
"""
# Create a Kernel object. No string encoding (all I/O is unicode)
kern = aiml.Kernel()
kern.setTextEncoding(None)
kern.bootstrap(learnFiles="OO_chatbot_project/mybot-basic.xml")


print("Welcome to this chat bot. Please feel free to ask questions from me!")
#######################################################
# Main loop
#######################################################
while True:
    #get user input
    try:
        userInput = input("> ")
    except (KeyboardInterrupt, EOFError) as e:
        print("Bye!")
        break
    #pre-process user input and determine response agent (if needed)
    responseAgent = 'aiml'
    #activate selected response agent
    if responseAgent == 'aiml':
        answer = kern.respond(userInput)
    if answer[0] == '#':
        params = answer[1:].split('$')
        cmd = int(params[0])
        if cmd == 0:
            print(params[1])
            break
        elif cmd == 1:
            try:
                details(params[1])
            except:
                print("Sorry, I do not know that. Be more specific!")
        elif cmd == 99:
            print("I did not get that, please try again :(.")
    else:
        print(answer)

Loading mybot-basic.xml...done (0.00 seconds)
Kernel bootstrap completed in 0.00 seconds
Welcome to this chat bot. Please feel free to ask questions from me!
> hi
Hello! How are you?
> i am good
I did not get that, please try again :(.
> i am fine
THATS COOL. HOW CAN I ASSIST YOU?
> who is donald trump
Sorry, I do not know that. Be more specific!
> bye
BYE! IT WAS GOOD TALKING TO YOU.
